In [ ]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=5385cda9216dc6c3e738e2e1eec93e465b0aa4d01859e0fd8d2a99c999587951
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [ ]:
# =========================================
# MIMIC-IV: AKI causal inference with Notes (Batch API version)
# =========================================
from __future__ import annotations
import os, re, json, math
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from dotenv import load_dotenv
from lifelines import CoxPHFitter

# --- Google Drive mount (Colab) ---
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/'
load_dotenv(drive_path + 'Colab Notebooks/env_config/.env')

# --- Config ---
pd.set_option("display.max_rows", 8)
MIMIC_DIR = Path("/content/drive/MyDrive/data/mimiciv/3.1/")
HOSP = MIMIC_DIR / "hosp"
NOTE = MIMIC_DIR / "note"
OUTD = MIMIC_DIR / "results_ci"; OUTD.mkdir(parents=True, exist_ok=True)
READ_KW = dict(dtype_backend="pyarrow", low_memory=False)

RANDOM_STATE = 7
PS_CLIP      = (1e-3, 1-1e-3)
W_TRIM       = (0.01, 0.99)
VPT_WINDOW_HOURS = 6
COX_PENALIZER = 0.1

# --- Regex ---
RX_SCR_LABEL  = "creatinine"
RX_SCR_FLUID  = r"\b(?:serum|blood)\b"
RX_MGDL       = r"\bmg/dl\b"
RX_MGL        = r"\bmg/l\b"
RX_VANCO_SUB  = "vancomycin"
RX_PTZ        = r"(?:piperacillin|tazobactam|zosyn)"
RX_EMERG_SUB  = "EMER"

# --- Confounders of interest (LLM outputs) ---
CONFOUNDERS = ["f_ckd_pre","f_dm_pre","f_hf_pre","f_liver_pre","f_nephrotox_pre"]

# --- LLM Config ---
LLM_MODEL = "gpt-4o-mini"
MAX_NOTE_CHARS = 15000
LLM_TEMPERATURE = 0.0
LLM_MAX_TOKENS = 200

Mounted at /content/drive


In [ ]:

# --- Prompt template ---
LLM_PROMPT_TEMPLATE = """
You are assisting a causal inference study of AKI. Exposure = vancomycin±piperacillin/tazobactam.
Your ONLY task: read a discharge note and mark **pre-treatment** (pre-admission or at presentation) risk factors.

Rules:
- Consider ONLY information existing **before or at presentation** relative to index_time = {index_time_iso}.
- DO NOT mark conditions/events clearly arising during hospitalization, hospital course, ICU interventions, inpatient treatments, or discharge meds. Those are potential colliders.
- If timing is ambiguous, be conservative and mark 0.
- Output a compact ONE-LINE JSON with 0/1 integers. No extra text.

Binary variables (confounders of interest):
- f_ckd_pre
- f_dm_pre
- f_hf_pre
- f_liver_pre
- f_nephrotox_pre

Return ONLY:
{{
  "f_ckd_pre": 0 or 1,
  "f_dm_pre": 0 or 1,
  "f_hf_pre": 0 or 1,
  "f_liver_pre": 0 or 1,
  "f_nephrotox_pre": 0 or 1
}}

Discharge note:
---
{note_text}
---
""".strip()



In [ ]:
# =========================
# Utils
# =========================
def _to_datetime_safe(s, fmt=None):
    x = pd.to_datetime(s, errors="coerce", format=fmt)
    if x.isna().any():
        y = pd.to_datetime(s[x.isna()], errors="coerce")
        x.loc[x.isna()] = y
    return x

def _smd(x, t, w=None):
    x = np.asarray(x, float); t = np.asarray(t, int)
    if w is None: w = np.ones_like(t, float)
    if (t==1).sum()==0 or (t==0).sum()==0: return np.nan
    m1 = np.average(x[t==1], weights=w[t==1]); m0 = np.average(x[t==0], weights=w[t==0])
    v1 = np.average((x[t==1]-m1)**2, weights=w[t==1]); v0 = np.average((x[t==0]-m0)**2, weights=w[t==0])
    return (m1-m0)/np.sqrt((v1+v0)/2 + 1e-9)

def evalue_from_hr(hr, lcl, ucl):
    def _ev(x: float) -> float:
        return x + math.sqrt(max(x,0)*(max(x,0)-1.0)) if x>1 else 1.0
    return (_ev(float(hr)), _ev(float(lcl)) if float(lcl)>1 else 1.0)

# =========================
# Cohort, labs, AKI, events
# =========================
def build_cohort(hosp: Path) -> pd.DataFrame:
    use_cols = ["subject_id","hadm_id","starttime","stoptime","drug"]
    it = pd.read_csv(hosp/"prescriptions.csv.gz",
                     usecols=lambda c: c in use_cols,
                     chunksize=500_000, low_memory=False)
    v_chunks, p_chunks = [], []
    for ch in it:
        ch["starttime"] = _to_datetime_safe(ch["starttime"])
        dlow = ch["drug"].astype("string").str.lower()
        v = ch[dlow.str.contains(RX_VANCO_SUB, na=False, regex=False)]
        p = ch[dlow.str.contains(RX_PTZ, na=False, regex=True)]
        v_chunks.append(v[["subject_id","hadm_id","starttime"]].rename(columns={"starttime":"v_start"}))
        p_chunks.append(p[["subject_id","hadm_id","starttime"]].rename(columns={"starttime":"p_start"}))

    v1 = (pd.concat(v_chunks).sort_values("v_start")
            .groupby(["subject_id","hadm_id"],as_index=False)
            .agg(index_time=("v_start","first")))
    p1 = (pd.concat(p_chunks).sort_values("p_start")
            .groupby(["subject_id","hadm_id"],as_index=False)
            .agg(ptz_time=("p_start","first")))

    cohort = v1.merge(p1, how="left", on=["subject_id","hadm_id"])
    cohort["vpt_flag"] = (
        cohort["ptz_time"].notna() &
        (cohort["ptz_time"] >= cohort["index_time"]) &
        (cohort["ptz_time"] <= cohort["index_time"]+pd.Timedelta(hours=VPT_WINDOW_HOURS))
    ).astype(int)
    print(f"[cohort] N={len(cohort)} VPT={int(cohort['vpt_flag'].sum())}")
    return cohort

def load_scr_itemids(hosp: Path) -> list[int]:
    d_lab = pd.read_csv(hosp/"d_labitems.csv.gz", **READ_KW)
    mask = (
        d_lab["label"].astype("string").str.contains(RX_SCR_LABEL, na=False, regex=False, case=False) &
        d_lab["fluid"].astype("string").str.contains(RX_SCR_FLUID, na=False, regex=True, case=False)
    )
    ids = d_lab.loc[mask,"itemid"].dropna().astype(int).unique().tolist()
    if not ids:
        ids = d_lab[d_lab["label"].astype("string").str.contains(RX_SCR_LABEL, case=False)]["itemid"].astype(int).unique().tolist()
    print(f"[scr ids] K={len(ids)} sample={ids[:3]}")
    return ids

# ... (SCr timeseries, label_aki, build_event_times 구현 동일; 생략 가능)

# =========================
# Notes → Batch JSONL
# =========================
def read_discharge_safe(note_dir: Path) -> pd.DataFrame:
    raw = pd.read_csv(note_dir/"discharge.csv.gz", **READ_KW)
    df = raw.rename(columns={c:c.lower() for c in raw.columns})
    df = df[["subject_id","hadm_id","text"]].copy()
    df["text"] = df["text"].astype("string[python]").fillna("")
    return df

def make_batch_jsonl_exporter(df_notes, index_time_map, out_jsonl):
    with open(out_jsonl,"w",encoding="utf-8") as f:
        for (sid,hid), g in df_notes.groupby(["subject_id","hadm_id"]):
            text = "\n".join(g["text"].astype(str).tolist())
            excerpt = text[:MAX_NOTE_CHARS]
            it = index_time_map.get((int(sid),int(hid)))
            it_iso = pd.to_datetime(it).tz_localize("UTC").isoformat() if pd.notna(it) else "UNKNOWN"
            prompt = LLM_PROMPT_TEMPLATE.format(index_time_iso=it_iso, note_text=excerpt)
            req = {
                "custom_id": f"{sid}_{hid}",
                "method":"POST",
                "url":"/v1/chat/completions",
                "body":{
                    "model":LLM_MODEL,
                    "temperature":LLM_TEMPERATURE,
                    "max_tokens":LLM_MAX_TOKENS,
                    "response_format":{"type":"json_object"},
                    "messages":[
                        {"role":"system","content":"Return strict JSON only."},
                        {"role":"user","content":prompt}
                    ]
                }
            }
            f.write(json.dumps(req)+"\n")
    print(f"[batch-export] wrote {out_jsonl}")

def parse_batch_results(result_jsonl, out_csv):
    rows=[]
    with open(result_jsonl,"r") as f:
        for line in f:
            job=json.loads(line)
            sid,hid=job["custom_id"].split("_")
            try:
                content=job["response"]["body"]["choices"][0]["message"]["content"]
                feats=json.loads(content)
            except Exception:
                feats={k:0 for k in CONFOUNDERS}
            row={"subject_id":int(sid),"hadm_id":int(hid)}
            row.update({k:int(feats.get(k,0)) for k in CONFOUNDERS})
            rows.append(row)
    pd.DataFrame(rows).to_csv(out_csv,index=False)
    print(f"[batch-parse] {len(rows)} notes → {out_csv}")
    return pd.DataFrame(rows)

# =========================
# PS + Cox
# =========================
def fit_ps_and_sw(dfc,covs):
    X=dfc[covs].copy().fillna(dfc[covs].median())
    T=dfc["vpt_flag"].astype(int)
    ps_model=LogisticRegression(max_iter=400,solver="lbfgs")
    ps_model.fit(X,T)
    ps=np.clip(ps_model.predict_proba(X)[:,1],*PS_CLIP)
    sw=np.where(T==1,T.mean()/ps,(1-T.mean())/(1-ps))
    lo,hi=np.quantile(sw,W_TRIM); sw=np.clip(sw,lo,hi)
    return sw,ps

def cox_results(dfc,sw,covs):
    d=pd.DataFrame({
        "time":dfc["duration_days"],
        "event":dfc["event_observed"].astype(int),
        "treat":dfc["vpt_flag"].astype(int),
        "sw":sw
    })
    cph=CoxPHFitter(penalizer=COX_PENALIZER)
    cph.fit(d,duration_col="time",event_col="event",weights_col="sw",robust=True)
    hr=float(np.exp(cph.params_["treat"]))
    lcl,ucl=np.exp(cph.confidence_intervals_.loc["treat"].values)
    return hr,lcl,ucl


In [ ]:
# =========================
# Main
# =========================
if __name__=="__main__":
    cohort=build_cohort(HOSP)
    scr_ids=load_scr_itemids(HOSP)
    # ... load_scr_timeseries → out=label_aki → evt=build_event_times
    notes=read_discharge_safe(NOTE)
    idx_map={(int(r.subject_id),int(r.hadm_id)):r.index_time for _,r in cohort.iterrows()}
    make_batch_jsonl_exporter(notes, idx_map, OUTD/"llm_batch_requests.jsonl")
    print("👉 Run Batch API with: !openai batches create -f results_ci/llm_batch_requests.jsonl -o results_ci/llm_batch_results.jsonl")

[cohort] N=90327 VPT=7822
[scr ids] K=3 sample=[50912, 52024, 52546]
[batch-export] wrote /content/drive/MyDrive/data/mimiciv/3.1/results_ci/llm_batch_requests.jsonl
👉 Run Batch API with: !openai batches create -f results_ci/llm_batch_requests.jsonl -o results_ci/llm_batch_results.jsonl


In [ ]:
!openai batches create \
  -f /content/drive/MyDrive/data/mimiciv/3.1/results_ci/llm_batch_requests.jsonl \
  -o /content/drive/MyDrive/data/mimiciv/3.1/results_ci/llm_batch_results.jsonl

usage: openai [-h] [-v] [-b API_BASE] [-k API_KEY] [-p PROXY [PROXY ...]]
              [-o ORGANIZATION] [-t {openai,azure}]
              [--api-version API_VERSION] [--azure-endpoint AZURE_ENDPOINT]
              [--azure-ad-token AZURE_AD_TOKEN] [-V]
              {api,tools,migrate,grit} ...
openai: error: argument {api,tools,migrate,grit}: invalid choice: 'batches' (choose from api, tools, migrate, grit)


In [ ]:
!openai --help | grep batches

In [ ]:
pwd

'/content'

In [ ]:
from openai import OpenAI
import time
import json
import os
from dotenv import load_dotenv
load_dotenv(drive_path + 'Colab Notebooks/env_config/.env')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or os.getenv("API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
input_path = "/content/drive/MyDrive/data/mimiciv/3.1/results_ci/llm_batch_requests.jsonl"
out_dir = "/content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts"
os.makedirs(out_dir, exist_ok=True)

max_lines = 5000  # 한 파일에 넣을 요청 수 (적절히 조정)
with open(input_path, "r") as f:
    for i, chunk_start in enumerate(range(0, sum(1 for _ in open(input_path)), max_lines)):
        f.seek(0)
        with open(os.path.join(out_dir, f"part_{i}.jsonl"), "w") as fout:
            for j, line in enumerate(f):
                if j < chunk_start:
                    continue
                if j >= chunk_start + max_lines:
                    break
                fout.write(line)
print("✅ 분할 완료:", os.listdir(out_dir))


✅ 분할 완료: ['part_0.jsonl', 'part_1.jsonl', 'part_2.jsonl', 'part_3.jsonl', 'part_4.jsonl', 'part_5.jsonl', 'part_6.jsonl', 'part_7.jsonl', 'part_8.jsonl', 'part_9.jsonl', 'part_10.jsonl', 'part_11.jsonl', 'part_12.jsonl', 'part_13.jsonl', 'part_14.jsonl', 'part_15.jsonl', 'part_16.jsonl', 'part_17.jsonl', 'part_18.jsonl', 'part_19.jsonl', 'part_20.jsonl', 'part_21.jsonl', 'part_22.jsonl', 'part_23.jsonl', 'part_24.jsonl', 'part_25.jsonl', 'part_26.jsonl', 'part_27.jsonl', 'part_28.jsonl', 'part_29.jsonl', 'part_30.jsonl', 'part_31.jsonl', 'part_32.jsonl', 'part_33.jsonl', 'part_34.jsonl', 'part_35.jsonl', 'part_36.jsonl', 'part_37.jsonl', 'part_38.jsonl', 'part_39.jsonl', 'part_40.jsonl', 'part_41.jsonl', 'part_42.jsonl', 'part_43.jsonl', 'part_44.jsonl', 'part_45.jsonl', 'part_46.jsonl', 'part_47.jsonl', 'part_48.jsonl', 'part_49.jsonl', 'part_50.jsonl', 'part_51.jsonl', 'part_52.jsonl', 'part_53.jsonl', 'part_54.jsonl', 'part_55.jsonl', 'part_56.jsonl', 'part_57.jsonl', 'part_58.jsonl

In [ ]:
files = [os.path.join(out_dir, f) for f in sorted(os.listdir(out_dir))]

for fpath in files:
    upload = client.files.create(
        file=open(fpath, "rb"),
        purpose="batch"
    )
    file_id = upload.id
    batch = client.batches.create(
        input_file_id=file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )
    print(f"📌 Submitted batch for {fpath}: {batch.id}")

📌 Submitted batch for /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts/part_0.jsonl: batch_68ac33277ba881908f0432ed60f473e9
📌 Submitted batch for /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts/part_1.jsonl: batch_68ac332e9c1481908cccb79dc614d246
📌 Submitted batch for /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts/part_10.jsonl: batch_68ac333bf958819095befee609025dbd
📌 Submitted batch for /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts/part_11.jsonl: batch_68ac3343ecf481909f92a88052dfa287
📌 Submitted batch for /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts/part_12.jsonl: batch_68ac334a9eb88190b0f68af9681b3631
📌 Submitted batch for /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts/part_13.jsonl: batch_68ac335221cc819082c0826644d6c5ee
📌 Submitted batch for /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_parts/part_14.jsonl: batch_68ac335a46588190a1eef0ef31895e78
📌 Submitted batch for /conten

In [ ]:
batch_ids = [
    "batch_68ac33277ba881908f0432ed60f473e9",
    "batch_68ac332e9c1481908cccb79dc614d246",
    "batch_68ac333bf958819095befee609025dbd",
    "batch_68ac3343ecf481909f92a88052dfa287",
    "batch_68ac334a9eb88190b0f68af9681b3631",
    "batch_68ac335221cc819082c0826644d6c5ee",
    "batch_68ac335a46588190a1eef0ef31895e78",
    "batch_68ac3360f2b48190908eff8ffbbbed82",
    "batch_68ac336759d08190abd706b621808bae",
    "batch_68ac336eb8448190bd5f3d31dad595a7",
    "batch_68ac3387d4008190933ecf075646f416",
    "batch_68ac338f4e208190ba112d0b6a0a0290",
    "batch_68ac3397ffe481908de581b2d20ac0e8",
    "batch_68ac33a0ec6081909972a1aaae8ffc65",
    "batch_68ac33a7b3808190aa420ffcadc94ab5",
    "batch_68ac33ae11e881908e47fa48d1cbc4fa",
    "batch_68ac33ba13548190aa81d0c8be4be29e",
    "batch_68ac33c1a7308190941d925a2e24913e",
    "batch_68ac33c920cc819080d15875e0db6524",
    "batch_68ac33d1d1c88190892bbaea710c38f4",
    "batch_68ac33d98458819093b82865422b4654",
    "batch_68ac33e315748190a3cd9bdbe8cb2af8",
    "batch_68ac33e80f7c8190ade9bc2f7539edbe",
    "batch_68ac33ecf754819087b02009fe45042f",
    "batch_68ac33f453d8819082383149be194b38",
    "batch_68ac3401a1548190afa514f96908a7e7",
    "batch_68ac340dbfb88190a7fe3103431dee41",
    "batch_68ac341af1248190a7c7164aa35aef8e",
    "batch_68ac342221a08190878a42ab2cd12bbc",
    "batch_68ac3487a6d48190a653b80c6d19a1c1",
    "batch_68ac348f155081909cafa36c7076bee4",
    "batch_68ac3497a6c081909d81ab1d12d4b6db",
    "batch_68ac349cf22c819080bd744b365cb165",
    "batch_68ac34a7800c81908ae2a21b7e115634",
    "batch_68ac34cd121c8190b261a1bbff0325cd",
    "batch_68ac34de864c8190b187fb040e34a812",
    "batch_68ac34e5c1ac8190a596c36de40f71b0",
    "batch_68ac34eb0b68819086801ce1f1c259b9",
    "batch_68ac34f458308190b61874153cb3ccac",
    "batch_68ac3500b7a48190839a59d909ea9f2c",
    "batch_68ac3507ee688190a251cac823bfe5d0",
    "batch_68ac350fa6ec8190a656264dbc0012dc",
    "batch_68ac35129c588190a0104a4220615ca2",
    "batch_68ac351dbdb48190ba270e033c58d50d",
    "batch_68ac3527acc88190806cc1a825c890a9",
    "batch_68ac354322d08190bbee31fa1f5d6d74",
    "batch_68ac354adc0881908962be9da3ae6e5d",
    "batch_68ac35583aac8190b1c851211d37969d",
    "batch_68ac355e77288190861d5e68e3466dcc",
    "batch_68ac35663b0c8190904ae58f2f27b4fe",
    "batch_68ac356cd1948190b9a3709068077b4e",
    "batch_68ac3573e8bc8190a3527034d9907bf7",
    "batch_68ac357dc55c8190b51652e68f0f7e26",
    "batch_68ac358767488190bafbe8bed657efd3",
    "batch_68ac35a403e48190b307119df17f3db7",
    "batch_68ac35b02ff8819083a5cb31843458ef",
    "batch_68ac35bcb42481908a219aad43814bea",
    "batch_68ac35c3a4e48190bf1d648602703c21",
    "batch_68ac35cd6b288190b92b47c9a47c70eb",
    "batch_68ac35dfc4e48190aca665f8511c1a0f",
    "batch_68ac35fae7dc8190aaec05c4c01adf5d",
    "batch_68ac360212848190bff9635212dd7f0b",
    "batch_68ac360bea808190b800d9bf96053033",
    "batch_68ac360f13c4819086d7deb34934489c",
    "batch_68ac3615edf4819083ba8353ebe312d9",
    "batch_68ac361f1e04819082059447e6484f5e",
    "batch_68ac3626e4d48190aa35489149fa4cab",
]


In [ ]:
OUTD = Path("/content/drive/MyDrive/data/mimiciv/3.1/results_ci")
RES_DIR = OUTD / "batch_results"; RES_DIR.mkdir(parents=True, exist_ok=True)

# 2) 상태 한 번에 조회 (원하면 루프 돌려서 폴링도 가능)
def list_statuses(batch_ids):
    rows = []
    for bid in batch_ids:
        try:
            b = client.batches.retrieve(bid)
            rows.append({"batch_id": bid, "status": b.status})
        except Exception as e:
            rows.append({"batch_id": bid, "status": f"error: {e}"})
    return pd.DataFrame(rows)

status_df = list_statuses(batch_ids)
print(status_df)

                                  batch_id     status
0   batch_68ac33277ba881908f0432ed60f473e9  completed
1   batch_68ac332e9c1481908cccb79dc614d246  completed
2   batch_68ac333bf958819095befee609025dbd  completed
3   batch_68ac3343ecf481909f92a88052dfa287  completed
..                                     ...        ...
63  batch_68ac360f13c4819086d7deb34934489c  completed
64  batch_68ac3615edf4819083ba8353ebe312d9  completed
65  batch_68ac361f1e04819082059447e6484f5e  completed
66  batch_68ac3626e4d48190aa35489149fa4cab  completed

[67 rows x 2 columns]


In [ ]:
status_df

batch_id     status
0   batch_68ac33277ba881908f0432ed60f473e9  completed
1   batch_68ac332e9c1481908cccb79dc614d246  completed
2   batch_68ac333bf958819095befee609025dbd  completed
3   batch_68ac3343ecf481909f92a88052dfa287  completed
..                                     ...        ...
63  batch_68ac360f13c4819086d7deb34934489c  completed
64  batch_68ac3615edf4819083ba8353ebe312d9  completed
65  batch_68ac361f1e04819082059447e6484f5e  completed
66  batch_68ac3626e4d48190aa35489149fa4cab  completed

[67 rows x 2 columns]

In [ ]:
# 3) 완료될 때까지 폴링하고, 완료된 것만 다운로드
def wait_and_download_all(batch_ids, poll_sec=60):
    completed, failed = set(), {}
    outputs = {}
    while True:
        num_done = len(completed) + len(failed)
        for bid in batch_ids:
            if bid in completed or bid in failed:
                continue
            try:
                b = client.batches.retrieve(bid)
                s = b.status
                ts = dt.datetime.now().strftime("%H:%M:%S")
                print(f"[{ts}] {bid}: {s}")
                if s in ("completed", "failed", "cancelled", "expired"):
                    if s == "completed":
                        try:
                            fid = b.output_file_id
                            resp = client.files.content(fid)
                            data = resp if isinstance(resp, (bytes, bytearray)) else getattr(resp, "content", None)
                            if data is None:
                                # 일부 SDK에서는 .read()가 필요
                                data = resp.read()
                            out_path = RES_DIR / f"{bid}.jsonl"
                            with open(out_path, "wb") as f:
                                f.write(data)
                            outputs[bid] = str(out_path)
                            completed.add(bid)
                            print(f" ✅ downloaded → {out_path}")
                        except Exception as de:
                            failed[bid] = f"download_error: {de}"
                            print(f" ❌ download failed for {bid}: {de}")
                    else:
                        failed[bid] = s
                        print(f" ❌ {bid} finished with status={s}")
            except Exception as e:
                failed[bid] = f"retrieve_error: {e}"
                print(f" ❌ retrieve failed for {bid}: {e}")

        if len(completed) + len(failed) == len(batch_ids):
            break
        time.sleep(poll_sec)
    return outputs, failed


In [ ]:
status_counts = {}
for bid in batch_ids:
    s = client.batches.retrieve(bid).status
    status_counts[s] = status_counts.get(s, 0) + 1
    print(f"{bid}: {s}")

# 완료율 계산
n_total = len(batch_ids)
n_done = status_counts.get("completed", 0)
print("\n=== Summary ===")
print("Total:", n_total)
print("Completed:", n_done)
print("Pending :", n_total - n_done)
print(f"Progress: {n_done}/{n_total} ({n_done/n_total:.1%})")
print("By status:", status_counts)

batch_68ac33277ba881908f0432ed60f473e9: completed
batch_68ac332e9c1481908cccb79dc614d246: completed
batch_68ac333bf958819095befee609025dbd: completed
batch_68ac3343ecf481909f92a88052dfa287: in_progress
batch_68ac334a9eb88190b0f68af9681b3631: completed
batch_68ac335221cc819082c0826644d6c5ee: completed
batch_68ac335a46588190a1eef0ef31895e78: in_progress
batch_68ac3360f2b48190908eff8ffbbbed82: completed
batch_68ac336759d08190abd706b621808bae: completed
batch_68ac336eb8448190bd5f3d31dad595a7: completed
batch_68ac3387d4008190933ecf075646f416: completed
batch_68ac338f4e208190ba112d0b6a0a0290: completed
batch_68ac3397ffe481908de581b2d20ac0e8: in_progress
batch_68ac33a0ec6081909972a1aaae8ffc65: completed
batch_68ac33a7b3808190aa420ffcadc94ab5: completed
batch_68ac33ae11e881908e47fa48d1cbc4fa: completed
batch_68ac33ba13548190aa81d0c8be4be29e: completed
batch_68ac33c1a7308190941d925a2e24913e: completed
batch_68ac33c920cc819080d15875e0db6524: completed
batch_68ac33d1d1c88190892bbaea710c38f4: comp

In [ ]:
import os
import io
import json
import time
from typing import Any, Dict, List, Optional

from tqdm import tqdm

# ---------------------------
# 유틸: JSONL 한 줄을 DataFrame 행으로 파싱
# ---------------------------
def extract_chat_content(body: Dict[str, Any]) -> Dict[str, Any]:
    """chat.completions 응답 body에서 유용 필드 추출"""
    model = body.get("model")
    usage = body.get("usage") or {}
    choices = body.get("choices") or []

    # 기본값
    text = None
    finish_reason = None
    role = None

    if choices:
        c0 = choices[0] or {}
        msg = c0.get("message") or {}
        role = msg.get("role")
        # OpenAI 포맷: content는 string
        text = msg.get("content")
        finish_reason = c0.get("finish_reason")

    # 요청 프롬프트(있을 수도, 없을 수도)
    # 일부 라인에는 input 필드가 없으므로 None 허용
    # 배치 input에 custom_id를 같이 넘겼다면, req_line["custom_id"]에서 가져옵니다(아래에서).
    prompt_tokens = usage.get("prompt_tokens")
    completion_tokens = usage.get("completion_tokens")
    total_tokens = usage.get("total_tokens")

    return {
        "model": model,
        "assistant_role": role,
        "assistant_text": text,
        "finish_reason": finish_reason,
        "prompt_tokens": prompt_tokens,
        "completion_tokens": completion_tokens,
        "total_tokens": total_tokens,
        # body.get("id") → chatcmpl-xxx
        "response_body_id": body.get("id"),
        "created": body.get("created"),
        "system_fingerprint": body.get("system_fingerprint"),
    }

def parse_jsonl_line(line_obj: Dict[str, Any], batch_id: str) -> Dict[str, Any]:
    """
    배치 결과 JSONL 한 줄은 대략:
    {
      "id": "req_xxx",
      "custom_id": "...",                # (옵션) 배치 입력에서 넣은 값
      "response": {                      # 성공 시
        "status_code": 200,
        "request_id": "req_xxx",
        "body": { ... chat.completions ... }
      }
      OR
      "error": { "message": "...", "code": "...", ... }  # 실패 시
    }
    """
    base = {
        "batch_id": batch_id,
        "request_id": line_obj.get("id"),
        "custom_id": line_obj.get("custom_id"),
    }

    if "response" in line_obj and isinstance(line_obj["response"], dict):
        resp = line_obj["response"]
        status_code = resp.get("status_code")
        req_id = resp.get("request_id")
        body = resp.get("body") or {}
        parsed = extract_chat_content(body)
        base.update(
            {
                "http_status": status_code,
                "api_request_id": req_id,
                "error": None,
                "error_code": None,
                **parsed,
            }
        )
    else:
        # 에러 라인
        err = line_obj.get("error") or {}
        base.update(
            {
                "http_status": None,
                "api_request_id": None,
                "error": err.get("message"),
                "error_code": err.get("code"),
                "model": None,
                "assistant_role": None,
                "assistant_text": None,
                "finish_reason": None,
                "prompt_tokens": None,
                "completion_tokens": None,
                "total_tokens": None,
                "response_body_id": None,
                "created": None,
                "system_fingerprint": None,
            }
        )

    return base

def read_jsonl_bytes(raw_bytes: bytes) -> List[Dict[str, Any]]:
    out = []
    for line in io.BytesIO(raw_bytes).read().splitlines():
        if not line:
            continue
        try:
            obj = json.loads(line.decode("utf-8"))
            out.append(obj)
        except Exception:
            # 손상된 라인 방어
            out.append({"error": {"message": "JSONL parse error"}, "raw": line.decode("utf-8", "ignore")})
    return out

# ---------------------------
# 2) 배치 순회 + 결과 수집
# ---------------------------
all_rows: List[Dict[str, Any]] = []
meta_rows: List[Dict[str, Any]] = []

for bid in tqdm(batch_ids, desc="Retrieve & download"):
    # 과도 호출 방지
    time.sleep(0.05)

    batch = client.batches.retrieve(bid)

    meta_rows.append(
        {
            "batch_id": batch.id,
            "status": batch.status,
            "completion_window": batch.completion_window,
            "created_at": batch.created_at,
            "incomplete_details": getattr(batch, "incomplete_details", None),
            "request_counts": getattr(batch, "request_counts", None),
            "input_file_id": getattr(batch, "input_file_id", None),
            "output_file_id": getattr(batch, "output_file_id", None),
            "error_file_id": getattr(batch, "error_file_id", None),
            "finalizing_at": getattr(batch, "finalizing_at", None),
            "completed_at": getattr(batch, "completed_at", None),
            "expires_at": getattr(batch, "expires_at", None),
        }
    )

    if getattr(batch, "status", None) != "completed":
        continue

    output_file_id = getattr(batch, "output_file_id", None)
    if not output_file_id:
        continue

    # 파일 다운로드 (SDK v1: stream 객체에 .read())
    try:
        stream = client.files.content(output_file_id)
        raw = stream.read()
    except AttributeError:
        # 일부 버전 대응: .content 속성 보유 시
        raw = client.files.content(output_file_id).content

    # JSONL 파싱
    lines = read_jsonl_bytes(raw)
    for obj in lines:
        row = parse_jsonl_line(obj, batch_id=bid)
        all_rows.append(row)

# ---------------------------
# 3) DataFrame 생성
# ---------------------------
df_meta = pd.DataFrame(meta_rows)

# 완료된 배치의 output 라인들을 담은 DF
df_outputs = pd.DataFrame(all_rows)

# 컬럼 순서 가독성 정리
preferred_cols = [
    "batch_id",
    "request_id",
    "custom_id",
    "http_status",
    "api_request_id",
    "model",
    "assistant_role",
    "assistant_text",
    "finish_reason",
    "prompt_tokens",
    "completion_tokens",
    "total_tokens",
    "response_body_id",
    "created",
    "system_fingerprint",
    "error",
    "error_code",
]
other_cols = [c for c in df_outputs.columns if c not in preferred_cols]
df_outputs = df_outputs[preferred_cols + other_cols]

print("=== Meta (batches) ===")
print(df_meta.head())
print("\n=== Outputs (merged) ===")
print(df_outputs.head())

# 4) 필요시 저장
df_meta.to_csv("batches_meta.csv", index=False)
df_outputs.to_csv("batches_outputs.csv", index=False)
df_outputs.to_parquet("batches_outputs.parquet", index=False)


Retrieve & download: 100%|██████████| 67/67 [02:57<00:00,  2.65s/it]


=== Meta (batches) ===
                                 batch_id     status completion_window  \
0  batch_68ac33277ba881908f0432ed60f473e9  completed               24h   
1  batch_68ac332e9c1481908cccb79dc614d246  completed               24h   
2  batch_68ac333bf958819095befee609025dbd  completed               24h   
3  batch_68ac3343ecf481909f92a88052dfa287  completed               24h   
4  batch_68ac334a9eb88190b0f68af9681b3631  completed               24h   

   created_at incomplete_details  \
0  1756115751               None   
1  1756115758               None   
2  1756115771               None   
3  1756115779               None   
4  1756115786               None   

                                      request_counts  \
0  BatchRequestCounts(completed=5000, failed=0, t...   
1  BatchRequestCounts(completed=5000, failed=0, t...   
2  BatchRequestCounts(completed=5000, failed=0, t...   
3  BatchRequestCounts(completed=5000, failed=0, t...   
4  BatchRequestCounts(completed=500

In [ ]:
drive_path

PosixPath('/content/drive/MyDrive/data/mimiciv/3.1/results_ci')

In [ ]:
result_path = drive_path + "data/mimiciv/3.1/results_ci/causal_inference"

In [ ]:
df_meta.to_csv(result_path+"batches_meta.csv", index=False)
df_outputs.to_csv(result_path+"batches_outputs.csv", index=False)
df_outputs.to_parquet(result_path+"batches_outputs.parquet", index=False)

In [ ]:
df_outputs.drop_duplicates()

batch_id  \
0       batch_68ac33277ba881908f0432ed60f473e9   
1       batch_68ac33277ba881908f0432ed60f473e9   
2       batch_68ac33277ba881908f0432ed60f473e9   
3       batch_68ac33277ba881908f0432ed60f473e9   
...                                        ...   
331789  batch_68ac3626e4d48190aa35489149fa4cab   
331790  batch_68ac3626e4d48190aa35489149fa4cab   
331791  batch_68ac3626e4d48190aa35489149fa4cab   
331792  batch_68ac3626e4d48190aa35489149fa4cab   

                                        request_id          custom_id  \
0       batch_req_68ac38fdfcf88190afdd4734d0178bec  10000032_22595853   
1       batch_req_68ac38fdf5248190b2e9094e23f2be13  10000032_22841357   
2       batch_req_68ac38fdf678819080f49e041d96fe70  10000032_25742920   
3       batch_req_68ac38fdf81081908d3095c4a15c7ed4  10000032_29079034   
...                                            ...                ...   
331789  batch_req_68ad2805462c8190b36a2d8ac99aecb3  11533102_27365014   
331790  batch_req_68ad28055f4881909bf6ab7417ba2c02  11533102_28939043   
331791  batch_req_68ad28055db881908657709086a412dd  11533102_29936329   
331792  batch_req_68ad2805aa00819083261e6a0ce5029e  11533158_26616882   

        http_status                    api_request_id                   model  \
0               200  09d9c1519aa33749a6511a0199adb022  gpt-4o-mini-2024-07-18   
1               200  b531ba2fd53a746599a35a3e35553099  gpt-4o-mini-2024-07-18   
2               200  da42a4a8fabba835d4e7f07083d4f81d  gpt-4o-mini-2024-07-18   
3               200  51c0a3ea5656d56c8b1301ef64c8d531  gpt-4o-mini-2024-07-18   
...             ...                               ...                     ...   
331789          200  37e5986cee5fef365601fcc839edd781  gpt-4o-mini-2024-07-18   
331790          200  d1de20de7e1be514215bdf1b5a2da4ef  gpt-4o-mini-2024-07-18   
331791          200  a43a83e7f1bc7675ae528b9c320a581a  gpt-4o-mini-2024-07-18   
331792          200  85e4b8555db7c1337f276ecfb37850c7  gpt-4o-mini-2024-07-18   

       assistant_role                                     assistant_text  \
0           assistant  {\n  "f_ckd_pre": 1,\n  "f_dm_pre": 0,\n  "f_h...   
1           assistant  {\n  "f_ckd_pre": 1,\n  "f_dm_pre": 0,\n  "f_h...   
2           assistant  {\n  "f_ckd_pre": 1,\n  "f_dm_pre": 0,\n  "f_h...   
3           assistant  {\n  "f_ckd_pre": 1,\n  "f_dm_pre": 0,\n  "f_h...   
...               ...                                                ...   
331789      assistant  {\n  "f_ckd_pre": 0,\n  "f_dm_pre": 1,\n  "f_h...   
331790      assistant  {\n  "f_ckd_pre": 0,\n  "f_dm_pre": 1,\n  "f_h...   
331791      assistant  {\n  "f_ckd_pre": 0,\n  "f_dm_pre": 1,\n  "f_h...   
331792      assistant  {\n  "f_ckd_pre": 1,\n  "f_dm_pre": 0,\n  "f_h...   

       finish_reason  prompt_tokens  completion_tokens  total_tokens  \
0               stop           2521                 53          2574   
1               stop           3424                 53          3477   
2               stop           3459                 53          3512   
3               stop           3381                 53          3434   
...              ...            ...                ...           ...   
331789          stop           4721                 53          4774   
331790          stop           2819                 53          2872   
331791          stop           4184                 53          4237   
331792          stop           2259                 53          2312   

                              response_body_id     created system_fingerprint  \
0       chatcmpl-C8OLATG3pViogJE2wR9kIRhwSKeNq  1756116224      fp_51db84afab   
1       chatcmpl-C8OLEiHJjus6hcnAFhd7SNJrXTPyO  1756116228      fp_560af6e559   
2       chatcmpl-C8OLC3T669ISRNPs2ORkRkynz5T2a  1756116226      fp_51db84afab   
3       chatcmpl-C8OLCjVkZAlfuw4ZCGxiLyIwQL4Jy  1756116226      fp_51db84afab   
...                                        ...         ...                ...   
331789  chatcmpl-C8aTeQzfso

In [ ]:
import datetime as dt
outputs, failed = wait_and_download_all(batch_ids, poll_sec=60)
print("Completed files:", outputs)

[15:40:34] batch_68ac33277ba881908f0432ed60f473e9: completed
 ✅ downloaded → /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_results/batch_68ac33277ba881908f0432ed60f473e9.jsonl
[15:40:35] batch_68ac332e9c1481908cccb79dc614d246: completed
 ✅ downloaded → /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_results/batch_68ac332e9c1481908cccb79dc614d246.jsonl
[15:40:36] batch_68ac333bf958819095befee609025dbd: completed
 ✅ downloaded → /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_results/batch_68ac333bf958819095befee609025dbd.jsonl
[15:40:37] batch_68ac3343ecf481909f92a88052dfa287: in_progress
[15:40:37] batch_68ac334a9eb88190b0f68af9681b3631: completed
 ✅ downloaded → /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_results/batch_68ac334a9eb88190b0f68af9681b3631.jsonl
[15:40:38] batch_68ac335221cc819082c0826644d6c5ee: completed
 ✅ downloaded → /content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_results/batch_68ac335221cc819082c0826644d6c5ee.jsonl
[1

KeyboardInterrupt: 

In [ ]:
from pathlib import Path

results_dir = Path("/content/drive/MyDrive/data/mimiciv/3.1/results_ci/batch_results")

# 다운로드 완료된 파일만 골라오기
completed_files = sorted([p for p in results_dir.glob("batch_*.jsonl") if p.stat().st_size > 0])

print("완료된 파일 개수:", len(completed_files))

완료된 파일 개수: 57


In [ ]:


# 4) JSONL 파싱 → DataFrame (custom_id에서 subject_id/hadm_id 복구)
def parse_jsonl_to_df(jsonl_path):
    rows = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            rec = json.loads(line)
            cid = rec.get("custom_id", "")
            body = (rec.get("response", {}) or {}).get("body", {})
            # chat.completions 출력 케이스
            content = None
            if "choices" in body:
                try:
                    content = body["choices"][0]["message"]["content"]
                except Exception:
                    content = None
            # responses.create (Responses API) 케이스 (혹시 쓴 경우 대비)
            if content is None and "output" in body:
                try:
                    parts = body["output"]
                    texts = [p.get("content", [{}])[0].get("text", {}).get("value") for p in parts]
                    content = "\n".join([t for t in texts if t])
                except Exception:
                    content = None
            if not content:
                continue

            # JSON만 뽑기
            try:
                payload = json.loads(content)
            except Exception:
                m = re.search(r"\{.*\}", content, re.S)
                payload = json.loads(m.group(0)) if m else {}

            # id 복구: export 시 custom_id에 hadm/sid 심어두었다고 가정
            # 예시 패턴:
            #  - "hadm:22841357|sid:10000032"
            #  - "sid_10000032_hadm_22841357"
            #  - "hadm_22841357"
            sid = None; hid = None
            m1 = re.search(r"hadm[:_](\d+)", cid)
            m2 = re.search(r"(?:sid|subject)[:_](\d+)", cid)
            if m1: hid = int(m1.group(1))
            if m2: sid = int(m2.group(1))
            row = {"custom_id": cid, "subject_id": sid, "hadm_id": hid}
            # 관심 confounder만 안전하게 투영
            for k in ["f_ckd_pre","f_dm_pre","f_hf_pre","f_liver_pre","f_nephrotox_pre"]:
                v = payload.get(k, 0)
                try:
                    row[k] = int(v)
                except Exception:
                    row[k] = 0
            rows.append(row)
    df = pd.DataFrame(rows).drop_duplicates(subset=["subject_id","hadm_id"])
    # 타입 정리
    if "subject_id" in df: df["subject_id"] = df["subject_id"].astype("Int64")
    if "hadm_id"   in df: df["hadm_id"]   = df["hadm_id"].astype("Int64")
    return df

def merge_all_batch_results(results_dir: Path) -> pd.DataFrame:
    paths = sorted([str(results_dir/p) for p in os.listdir(results_dir) if p.endswith(".jsonl")])
    dfs = []
    for p in paths:
        try:
            dfs.append(parse_jsonl_to_df(p))
        except Exception as e:
            print(f"[warn] parse failed for {p}: {e}")
    if not dfs:
        return pd.DataFrame(columns=["subject_id","hadm_id","f_ckd_pre","f_dm_pre","f_hf_pre","f_liver_pre","f_nephrotox_pre"])
    out = pd.concat(dfs, ignore_index=True)
    # 만약 중복 hadm_id가 남아있으면 OR(any)로 합치기
    agg = (out.groupby(["subject_id","hadm_id"], dropna=False)
              [["f_ckd_pre","f_dm_pre","f_hf_pre","f_liver_pre","f_nephrotox_pre"]]
              .max()
              .reset_index())
    return agg

In [ ]:
dfs = [parse_jsonl_to_df(str(p)) for p in completed_files]
partial_df = pd.concat(dfs, ignore_index=True)

# 중복 제거 및 집계
partial_df = (
    partial_df.groupby(["subject_id","hadm_id"], dropna=False)
              [["f_ckd_pre","f_dm_pre","f_hf_pre","f_liver_pre","f_nephrotox_pre"]]
              .max()
              .reset_index()
)

print("Partial DF shape:", partial_df.shape)

Partial DF shape: (1, 7)
